In [34]:
import pandas as pd

In [35]:
from googleapiclient.discovery import build

# Load your API key
with open("api_key.txt", "r") as f:
    api_key = f.read().strip()

# Create the YouTube API client
youtube = build('youtube', 'v3', developerKey=api_key)

In [36]:
def get_channel_stats(youtube, channel_id):
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=channel_id
    )
    response = request.execute()

    if 'items' not in response or len(response['items']) == 0:
        print(f"❌ No data found for Channel ID: {channel_id}")
        return None

    uploads_playlist = response['items'][0]['contentDetails']['relatedPlaylists']['uploads']

    data = {
        "Channel Name": response['items'][0]['snippet']['title'],
        "Subscribers": int(response['items'][0]['statistics']['subscriberCount']),
        "Total Views": int(response['items'][0]['statistics']['viewCount']),
        "Total Videos": int(response['items'][0]['statistics']['videoCount']),
        "Uploads Playlist": uploads_playlist
        
    }

    return data

# add channel id here
channel_ids = [
    "UCX6OQ3DkcsbYNE6H8uQQuVA",    #mrbeast channelid
    
  
 ]  

all_data = []

for cid in channel_ids:
    stats = get_channel_stats(youtube, cid)
    all_data.append(stats)

df = pd.DataFrame(all_data)
print(df)


HttpError: <HttpError 400 when requesting https://youtube.googleapis.com/youtube/v3/channels?part=snippet%2CcontentDetails%2Cstatistics&id=UCX6OQ3DkcsbYNE6H8uQQuVA&key=...&alt=json returned "API key not valid. Please pass a valid API key.". Details: "[{'message': 'API key not valid. Please pass a valid API key.', 'domain': 'global', 'reason': 'badRequest'}]">

In [ ]:
def get_video_ids(youtube, playlist_id, max_results=50):
    video_ids = []
    request = youtube.playlistItems().list(
        part="contentDetails",
        playlistId=playlist_id,
        maxResults=max_results
    )
    response = request.execute()
    for item in response["items"]:
        video_ids.append(item["contentDetails"]["videoId"])
    return video_ids


In [ ]:
def get_video_details(youtube, video_ids):
    all_details = []
    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part="snippet,statistics",
            id=",".join(video_ids[i:i+50])
        )
        response = request.execute()
        for video in response["items"]:
            details = {
                "Title": video["snippet"]["title"],
                "Views": int(video["statistics"].get("viewCount", 0)),
                "Likes": int(video["statistics"].get("likeCount", 0)),
                "Comments": int(video["statistics"].get("commentCount", 0))
            }
            all_details.append(details)
    return all_details


In [ ]:
import pandas as pd


channel_id =  "UCX6OQ3DkcsbYNE6H8uQQuVA"


channel_stats = get_channel_stats(youtube, channel_id)
print(" Channel Stats:\n", channel_stats)


video_ids = get_video_ids(youtube, channel_stats["Uploads Playlist"])

videos_data = get_video_details(youtube, video_ids)


df_videos = pd.DataFrame(videos_data)


most_viewed = df_videos.sort_values(by="Views", ascending=False).head(1)
most_liked = df_videos.sort_values(by="Likes", ascending=False).head(1)



print("\n Most Viewed Video:")
print(most_viewed)
print("\n Most Liked Video:")
print(most_liked)

print("\n Average Views per Video:", df_videos['Views'].mean())
print("\n Average Likes per Video:", df_videos['Likes'].mean())
print("\n Average Comments per Video:", df_videos['Comments'].mean())


 Channel Stats:
 {'Channel Name': 'MrBeast', 'Subscribers': 386000000, 'Total Views': 80106899132, 'Total Videos': 862, 'Uploads Playlist': 'UUX6OQ3DkcsbYNE6H8uQQuVA'}

 Most Viewed Video:
                                 Title      Views     Likes  Comments
45  Will A Guitar Boat Hold My Weight?  654207500  17432143     14470

 Most Liked Video:
                                 Title      Views     Likes  Comments
45  Will A Guitar Boat Hold My Weight?  654207500  17432143     14470

 Average Views per Video: 220465047.66

 Average Likes per Video: 7018984.42

 Average Comments per Video: 29653.98
